# Landscape of Vibrational Anharmonicity Scores for Cubic Perovskites


## The original definition of anharmonicity score $\sigma(T)$

This first part of the notebook provides you the code to analyze the vibrational anharmonicity scores of cubic perovskites at $T=$300 K with respect to either their 'convex-hull' stabilities $\Delta H_c$, or their Goldschmidt tolerace factors. Practically, as shown in Notebook #1, the time-dependent vibrational anharmonicity scores $\sigma(T)$ is calculated for every frame of the *ab initio* MD trajectory [to be denoted as $\sigma(T,t)$]. Here, the $\sigma(T)$ values chartted for each cubic perovskites will be the $\sigma(T,t)$ values ***averaged across the entire MD trajectory***.

In [1]:
from tolerance_factor import tolerance_factor
from chemical_space import *
from ase.db import connect

In [2]:
X='F'

#get the corresponding A, B cations that go with the anion
if X in halide_C:
    A = halide_A
    B = halide_B
elif X in chalco_C:
    A = chalco_A
    B = chalco_B

#connect to the corresponding database toto retrieve the data
db=connect('./perovskites_'+str(X)+'.db')

In [3]:
tolerance_factors=[]
energy_differences=[] #as the formation energies of Pm3m perovskite to the most stable random structure
label=[]
sigmas=[]
sigmas_3=[]
sigmas_300K_tdep=[]
sigmas_300K_4th_tdep_2=[]
sigmas_300K_4th_tdep_3=[]
sigmas_300K_4th_tdep_4=[]

for a in A:
    for b in B:
        row = None
        system_name = a + b + X
        uid = system_name + '_Pm3m'
        pm3m_formation_e = None
        
        tolerance_f = tolerance_factor(a, b, X)
        
        #Get the database entry corresponding to this cubic perovskite
        try:
            row = db.get(selection=[('uid', '=', uid)])
        except:
            continue
            
        if row is not None:
            #formation energy for the cubic perovskite
            try:
                pm3m_formation_e = row.key_value_pairs['formation_energy']
            except KeyError:
                continue
                
            sigma = None
            try:
                sigma = row.key_value_pairs['sigma_300K_single']
            except:
                pass
                
                
            sigma_3 = None
            try:
                sigma_3 = row.key_value_pairs['sigma_300K_third_order']
            except:
                pass
                
            sigma_300K_tdep = None
            try:
                sigma_300K_tdep = row.key_value_pairs['sigma_300K_tdep']
            except:
                pass
            
            sigma_300K_4th_tdep_2=None
            sigma_300K_4th_tdep_3=None
            sigma_300K_4th_tdep_4=None
            
            try:
                sigma_300K_4th_tdep_2 = row.key_value_pairs['sigma_300K_4th_tdep_2']
            except:
                pass
            
            try:
                sigma_300K_4th_tdep_3 = row.key_value_pairs['sigma_300K_4th_tdep_3']
            except:
                pass
            
            try:
                sigma_300K_4th_tdep_4 = row.key_value_pairs['sigma_300K_4th_tdep_4']
            except:
                pass
                
            if pm3m_formation_e is not None:
                #Get the formation energies for the most stable structure that is optimized from a random starting 
                #point. In the actual computational screening procedure, 10 structures were screened for each compound.
                #However, due to the storage limit of the Github repository, only the information on the 
                #lowest energy structure is stored. This bit of the code is trying to digging that information out
                random_energy = None
                counter = 0
                
                while (random_energy is None) or (counter<10):
                    uid_r = uid + '_rand_str_' + str(counter)
                    try:
                        row = db.get(selection=[('uid', '=', uid_r)])
                    except:
                        pass
                    
                    if row is not None:
                        random_energy = row.key_value_pairs['formation_energy']
                    counter+=1

                if (random_energy is not None) and (sigma is not None):
                    energy_differences.append(pm3m_formation_e - random_energy)
                    tolerance_factors.append(tolerance_f)
                    sigmas.append(sigma)
                    sigmas_3.append(sigma_3)
                    sigmas_300K_tdep.append(sigma_300K_tdep)
                    sigmas_300K_4th_tdep_2.append(sigma_300K_4th_tdep_2)
                    sigmas_300K_4th_tdep_3.append(sigma_300K_4th_tdep_3)
                    sigmas_300K_4th_tdep_4.append(sigma_300K_4th_tdep_4)
                    label.append(system_name+'3')

print("Total number of available data: "+str(len(energy_differences)))

Total number of available data: 491


In [4]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Div
from bokeh.models.tools import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.layouts import row as bokeh_row

output_notebook()

data = {'x1':tolerance_factors,
        'x2':energy_differences,
        'y':sigmas,
        'label':label}
source = ColumnDataSource(data=data)

p1 = figure(width=600, height=600, background_fill_color="#fafafa")
p1.circle(x='x1', y='y', source=source,size=10,color='#CB0000',alpha=0.6)
hover = HoverTool()
hover.tooltips=[
    ('t', '@x1'),
    ('sigma', '@y'),
    ('Compound', '@label')
]
p1.add_tools(hover)

p1.xaxis.axis_label='Tolerance Factor'
p1.yaxis.axis_label="Sigma (300 K)"
p1.xaxis.axis_label_text_font_size = "20pt"
p1.yaxis.axis_label_text_font_size = "16pt"
p1.xaxis.major_label_text_font_size = "15pt"
p1.yaxis.major_label_text_font_size = "15pt"

#==========================================================

p2 = figure(width=600, height=600, background_fill_color="#fafafa")
p2.circle(x='x2', y='y', source=source,size=10,color='#FFBB00',alpha=0.6)
hover = HoverTool()
hover.tooltips=[
    ('Delta Hc', '@x2'),
    ('sigma', '@y'),
    ('Compound', '@label')
]
p2.add_tools(hover)

p2.xaxis.axis_label='Convex Hull Energies (eV/atom)'
p2.yaxis.axis_label="Sigma (300 K)"
p2.xaxis.axis_label_text_font_size = "20pt"
p2.yaxis.axis_label_text_font_size = "16pt"
p2.xaxis.major_label_text_font_size = "15pt"
p2.yaxis.major_label_text_font_size = "15pt"

show(bokeh_row(p1,p2))

Loading BokehJS ...

## Dissecting Higher-Order Contributions to Vibrational Anharmonicity

The large values of $\sigma$ observed for cubic perovskites may be attributed to the strong vibrational anharmonicities associated with this type of materials, particularly the contributions from the quartic order of force constants (FC). To reveal such an effect, recall that the Hamiltonian describing the ionic vibration written in Einstein notation can be written as 

$$\mathcal{H}=U_{0}+\mathbf{\Phi}_{i}^{\alpha}u_{i}^{\alpha}+\frac{1}{2!}\mathbf{\Phi}_{ij}^{\alpha\beta}u_{i}^{\alpha}u_{j}^{\beta}
    +\frac{1}{3!}\mathbf{\Phi}_{ijk}^{\alpha\beta\gamma}u_{i}^{\alpha}u_{j}^{\beta}u_{k}^{\gamma}+\frac{1}{4!}\mathbf{\Phi}_{ijkl}^{\alpha\beta\gamma\delta}u_{i}^{\alpha}u_{j}^{\beta}u_{k}^{\gamma}u_{l}^{\delta}+\mathcal{O}(u^{5})$$
    
where $\mathbf{\Phi}$ is the FC and  $u_{i}^{\alpha}$ is the displacement for $i$-th atom along the $\alpha$-Cartesian direction. With this, the corresponding atomic force is:

$$F_{i}^{\alpha}=F_{i}^{\alpha(2)}+F_{i}^{\alpha(3)}+F_{i}^{\alpha(4)}+\mathcal{O}(F_{i}^{\alpha(5)})=-\mathbf{\Phi}_{ij}^{\alpha\beta}u_{j}^{\beta}-\frac{1}{2}\mathbf{\Phi}_{ijk}^{\alpha\beta\gamma} u_{j}^{\beta} u_{k}^{\gamma}-\frac{1}{6}\mathbf{\Phi}_{ijkl}^{\alpha\beta\gamma\delta}u_{j}^{\beta}u_{k}^{\gamma}u_{l}^{\delta}+\mathcal{O}(u^{4}).$$

With this, we can gradually takes out the contributions from higher--order atomic forces and examine how the vibrational anharmonicity scores changes accordingly. Inspired by the above expressions, and rewrote the original definition of $\sigma$ as $\sigma^{(2)}$, we  can extend $\sigma^{(2)}$ to $\sigma^{(n)}$, by replacing the anharmonic forces $F_{i}^{\alpha,A}$ in its definition with $\mathcal{F}_{i}^{\alpha,(n)}$, defined as
$$\mathcal{F}_{i}^{\alpha,(n)}=F_{i}^{\alpha}-\sum_{\eta=2}^{n}F_{i}^{\alpha(\eta)}.$$ 

### Cubic Contributions with  Force-Constants Calculated with Finite-Displacement Approach

Here, we shows the relationship between $\sigma^{(3)}$ and $\sigma^{(2)}$, in which the third-order FC $\mathbf{\Phi}_{ijk}^{\alpha\beta\gamma}$ for each perovskite is calculated using the same finite-displacement approach as for the second-order FC $\mathbf{\Phi}_{ij}^{\alpha\beta}$.

In [5]:
_sigmas=[]
_sigmas_3=[]
_labels=[]

for c,s in enumerate(sigmas_3):
    if s is not None:
        _sigmas.append(sigmas[c])
        _sigmas_3.append(s)
        _labels.append(label[c])

data_2_3 = {'x':_sigmas,
        'y':_sigmas_3,
        'label':_labels}
source_2_3 = ColumnDataSource(data=data_2_3)

if X in chalco_C:
    ranges=[0.1,50]
elif X in halide_C:
    ranges=[0.09,10]
p_23 = figure(width=600, height=600, background_fill_color="#fafafa",x_axis_type="log",y_axis_type="log"
                                                                    ,x_range=ranges,y_range=ranges)
p_23.circle(x='x', y='y', source=source_2_3,size=10,color='#CB0000',alpha=0.6)
p_23.line([min(_sigmas)-0.5*min(_sigmas),max(_sigmas)],[min(_sigmas),max(_sigmas)],color='black',width=2,line_dash=[6])

p_23.xaxis.axis_label='sigma(2)'
p_23.yaxis.axis_label="sigma(3)"
p_23.xaxis.axis_label_text_font_size = "20pt"
p_23.yaxis.axis_label_text_font_size = "16pt"
p_23.xaxis.major_label_text_font_size = "15pt"
p_23.yaxis.major_label_text_font_size = "15pt"

hover = HoverTool()
hover.tooltips=[
    ('sigma_2', '@x'),
    ('sigma_3', '@y'),
    ('Compound', '@label')
]
p_23.add_tools(hover)
show(p_23)

### Including Cubic and Quartic Contributions with Temperature-Dependent Effective Potentials (TDEP)

According to the theory of TDEP, the structures are not stable when an atom is moving in the potential of its stationary neighbours, but stabilised by moving in the effective potential of nonstationary neighbours. These nonstationary states can be obtained from *ab initio* MD sampling. The effective force constants $\tilde{\mathbf{\Phi}}^{(p,q)}$ may be obtained by using a supervised machine learning approach through minimizing the error function:

$$\min_{\tilde{\mathbf{\Phi}}}\left\vert\left\vert F_{i}^{\alpha} - \left[\sum_{p=2}^{q} -\tilde{\mathbf{\Phi}}^{(p,q)}u_{j}^{\beta}\cdots u_{l}^{\delta}\right] \right\vert\right\vert$$

in which $F_{i}^{\alpha}$ and $u$ are atomic forces and displacements obtained from *ab initio* MD. $q$ represents the highest order at which the expansion will be truncated, this will give $q-1$ effective force constants from the fitting.  Replacing $\mathbf{\Phi}$ by  $\tilde{\mathbf{\Phi}}$ in the expression of the atomic forces leads to the total forces $F_{i}^{\alpha}=-\sum_{p=2}^{q}\tilde{\mathbf{\Phi}}^{(p,q)}u_{j}^{\beta}\cdots u_{l}^{\delta}+\mathcal{O}(u^{q})=\sum_{p=2}^{q}\tilde{F}_{i}^{\alpha(p,q)}+\mathcal{O}(u^{q})$, in which we denoted $\tilde{F}_{i}^{\alpha(p,q)}$ being the $p$-th order temperature-dependent effective force when the expansion is truncated at the $q$-th order. From here, given a specific $q$, the higher-order $(>p)$ anharmonic contribution to the toal atomic force is given by

$$\tilde{\mathcal{F}}_{i}^{\alpha(p,q)}=F_{i}^{\alpha}-\sum_{\eta=2}^{p}\tilde{F}_{i}^{\alpha(\eta,q)}$$

Replacing $F_{i}^{\alpha,A}$ by $\tilde{\mathcal{F}}_{i}^{\alpha(p,q)}$ in the definition of the anharmonicity score  gives us a temperature--dependent effective anharmonic score, which will be dubbed as $\tilde{\sigma}^{(p,q)}$. In the subsequent part of this notebook, we will examine the behaviours of $\tilde{\sigma}^{(p,q)}$ by focuing on the cases where $q=2$ and $q=4$.

### Absorbing all higher-order contributions to an effective harmonic force constant $\tilde{\mathbf{\Phi}}^{(2,2)}$ which leads to $\tilde{\sigma}^{(2,2)}$

In [6]:
_sigmas=[]
_sigmas_300K_tdep=[]
_labels=[]

for c,s in enumerate(sigmas_300K_tdep):
    if (s is not None) and (sigmas[c] is not None):
        _sigmas.append(sigmas[c])
        _sigmas_300K_tdep.append(s)
        _labels.append(label[c])

data_2_2_tdep = {'x':_sigmas,
                 'y':_sigmas_300K_tdep,
                 'label':_labels}
source_2_2_tdep = ColumnDataSource(data=data_2_2_tdep)

if X in chalco_C:
    ranges=[0.1,50]
elif X in halide_C:
    ranges=[0.09,10]
p_2_2_tdep = figure(width=600, height=600, background_fill_color="#fafafa",x_axis_type="log",y_axis_type="log"
                                                                    ,x_range=ranges,y_range=[0.3,1])
p_2_2_tdep.circle(x='x', y='y', source=source_2_2_tdep,size=10,color='#CB0000',alpha=0.6)

p_2_2_tdep.line([min(_sigmas)-0.5*min(_sigmas),max(_sigmas)],[min(_sigmas),max(_sigmas)],color='black',width=2,line_dash=[6])

p_2_2_tdep.xaxis.axis_label='sigma(2)'
p_2_2_tdep.yaxis.axis_label="sigma_tilde(2,2)"
p_2_2_tdep.xaxis.axis_label_text_font_size = "20pt"
p_2_2_tdep.yaxis.axis_label_text_font_size = "16pt"
p_2_2_tdep.xaxis.major_label_text_font_size = "15pt"
p_2_2_tdep.yaxis.major_label_text_font_size = "15pt"

hover = HoverTool()
hover.tooltips=[
    ('sigma_2', '@x'),
    ('sigma_(2,2)', '@y'),
    ('Compound', '@label')
]
p_2_2_tdep.add_tools(hover)
show(p_2_2_tdep)

### Including quartic force constants $\tilde{\mathbf{\Phi}}^{(p,4)}$ which leads to $\tilde{\sigma}^{(p,4)}$

In [7]:
_sigmas=[]
_sigmas_300K_4th_tdep_2=[]
_sigmas_300K_4th_tdep_3=[]
_sigmas_300K_4th_tdep_4=[]
_labels=[]

for c,s in enumerate(sigmas_300K_4th_tdep_2):
    if (s is not None) and (sigmas[c] is not None):
        _sigmas.append(sigmas[c])
        _sigmas_300K_4th_tdep_2.append(s)
        _sigmas_300K_4th_tdep_3.append(sigmas_300K_4th_tdep_3[c])
        _sigmas_300K_4th_tdep_4.append(sigmas_300K_4th_tdep_4[c])
        _labels.append(label[c])

data_tdep = {'x':_sigmas,
             'y_4_2':_sigmas_300K_4th_tdep_2,
             'y_4_3':_sigmas_300K_4th_tdep_3,
             'y_4_4':_sigmas_300K_4th_tdep_4,
             'label':_labels}
source_tdep = ColumnDataSource(data=data_tdep)

if X in chalco_C:
    ranges=[0.1,50]
elif X in halide_C:
    ranges=[0.09,10]
    
p1 = figure(width=500, height=500, background_fill_color="#fafafa",x_axis_type="log",y_axis_type="log"
                                                                    ,x_range=ranges,y_range=[0.3,3])
p1.circle(x='x', y='y_4_2', source=source_tdep,size=10,color='#ffc13b',alpha=0.5)

p1.line([min(_sigmas)-0.5*min(_sigmas),max(_sigmas)],[min(_sigmas),max(_sigmas)],color='black',width=2,line_dash=[6])

p1.xaxis.axis_label='sigma(2)'
p1.yaxis.axis_label="sigma_tilde(2,4)"
p1.xaxis.axis_label_text_font_size = "20pt"
p1.yaxis.axis_label_text_font_size = "16pt"
p1.xaxis.major_label_text_font_size = "15pt"
p1.yaxis.major_label_text_font_size = "15pt"

hover = HoverTool()
hover.tooltips=[
    ('sigma_2', '@x'),
    ('sigma_(2,4)', '@y_4_2'),
    ('Compound', '@label')
]
p1.add_tools(hover)

p2 = figure(width=500, height=500, background_fill_color="#fafafa",x_axis_type="log",y_axis_type="log"
                                                                    ,x_range=ranges,y_range=[0.3,3])
p2.circle(x='x', y='y_4_2', source=source_tdep,size=10,color='#ffc13b',alpha=0.25)
p2.circle(x='x', y='y_4_3', source=source_tdep,size=10,color='#CB0000',alpha=0.7)

p2.line([min(_sigmas)-0.5*min(_sigmas),max(_sigmas)],[min(_sigmas),max(_sigmas)],color='black',width=2,line_dash=[6])

p2.xaxis.axis_label='sigma(2)'
p2.yaxis.axis_label="sigma_tilde(3,4)"
p2.xaxis.axis_label_text_font_size = "20pt"
p2.yaxis.axis_label_text_font_size = "16pt"
p2.xaxis.major_label_text_font_size = "15pt"
p2.yaxis.major_label_text_font_size = "15pt"

hover = HoverTool()
hover.tooltips=[
    ('sigma_2', '@x'),
    ('sigma_(2,4)', '@y_4_2'),
    ('sigma_(3,4)', '@y_4_3'),
    ('Compound', '@label')
]
p2.add_tools(hover)


p3 = figure(width=500, height=500, background_fill_color="#fafafa",x_axis_type="log",y_axis_type="log"
                                                                    ,x_range=ranges,y_range=[0.3,3])
p3.circle(x='x', y='y_4_2', source=source_tdep,size=10,color='#ffc13b',alpha=0.25)
p3.circle(x='x', y='y_4_3', source=source_tdep,size=10,color='#CB0000',alpha=0.25)
p3.circle(x='x', y='y_4_4', source=source_tdep,size=10,color='#1e3d59',alpha=0.7)


p3.line([min(_sigmas)-0.5*min(_sigmas),max(_sigmas)],[min(_sigmas),max(_sigmas)],color='black',width=2,line_dash=[6])

p3.xaxis.axis_label='sigma(2)'
p3.yaxis.axis_label="sigma_tilde(4,4)"
p3.xaxis.axis_label_text_font_size = "20pt"
p3.yaxis.axis_label_text_font_size = "16pt"
p3.xaxis.major_label_text_font_size = "15pt"
p3.yaxis.major_label_text_font_size = "15pt"

hover = HoverTool()
hover.tooltips=[
    ('sigma_2', '@x'),
    ('sigma_(2,4)', '@y_4_2'),
    ('sigma_(3,4)', '@y_4_3'),
    ('sigma_(4,4)', '@y_4_4'),
    ('Compound', '@label')
]
p3.add_tools(hover)


show(bokeh_row(p1,p2,p3))